In [10]:
import os 
import sys
while os.path.split(os.getcwd())[1] != 'RecSysChallenge2023-Team':
    os.chdir('..')
sys.path.insert(1, os.getcwd())

################################# IMPORT HyperTuning  #################################
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from skopt.space import Real, Integer, Categorical

################################# IMPORT RECOMMENDERS #################################
from Recommenders.KNN.ItemKNN_CFCBF_Hybrid_Recommender import ItemKNN_CFCBF_Hybrid_Recommender
from Recommenders.IR_feature_weighting import okapi_BM_25

################################## IMPORT LIBRARIES ##################################
import scipy.sparse as sps
from tqdm import tqdm
import pandas as pd
import numpy as np
import similaripy
import math 

import Daniele.Utils.MatrixManipulation as mm
import MyDataManager as dm

from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from sklearn.preprocessing import normalize

URMv = dm.getURMviews()
URMo = dm.getURMopen()
ICMt = dm.getICMt()
ICMl = dm.getICMl()

URMv_train, URMv_test = split_train_in_two_percentage_global_sample(URMv, train_percentage = 0.90)
URMo_train, URMo_test = split_train_in_two_percentage_global_sample(URMo, train_percentage = 0.90)

urm =  mm.defaultExplicitURM(URMv_train,URMo_train,add_aug=True,normalize =False)


#urma = mm.augmentURM(mm.explicitURM(URM_train_validation))
#urm = sps.vstack([URM_train_validation,urma]).tocsr()
#urm = similaripy.normalization.bm25plus(urm)
    

icm = mm.augmentedICM(ICMt, ICMl)

recommender = ItemKNN_CFCBF_Hybrid_Recommender(urm,icm)
recommender.fit(topK= 674, shrink= 613, similarity= 'cosine', normalize= True, feature_weighting='TF-IDF')


100%|██████████| 597033/597033 [00:00<00:00, 1759130.36it/s]


ItemKNN_CFCBF_HybridRecommender: URM Detected 3461 (12.4%) items with no interactions.
Similarity column 27968 (100.0%), 2915.43 column/sec. Elapsed time 9.59 sec


In [11]:
evaluator_test = EvaluatorHoldout(URMv_test+URMo_test, [10])
ris, _ = evaluator_test.evaluateRecommender(recommender)
ris

EvaluatorHoldout: Ignoring 2676 ( 6.4%) Users that have less than 1 test interactions


/Users/daniele/Desktop/RecSys/RecSysChallenge2023-Team/Evaluation/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),
/Users/daniele/Desktop/RecSys/RecSysChallenge2023-Team/Evaluation/metrics.py:270: RuntimeWarning: invalid value encountered in double_scalars
  ndcg_ = rank_dcg / ideal_dcg


EvaluatorHoldout: Processed 38953 (100.0%) in 28.82 sec. Users per second: 1352


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.026596,0.060126,0.058691,0.012112,0.027347,0.096185,NaN,0.036605,0.210176,0.107602,...,0.935718,0.196666,0.935718,0.023219,8.364931,0.990015,0.053232,0.622567,4.464127,0.257577
